## Overview of GTEx Resources: working with gene expression, eQTL, and metadata files


This notebook illustrates how to parse the GTEx data tables and resources available from the [GTEx Portal](https://gtexportal.org), including:
* Sample and subject attributes
* Expression tables
* eQTL results

All examples use data from the V7 release.

#### Pre-requisites

**_For the online version of this notebook, all datasets used in the examples are available to the notebook, and no additional files are required._**

To run the notebook elsewhere, the following files are needed (all are available from the [downloads](https://gtexportal.org/home/datasets) section of the portal):
* GTEx_v7_Annotations_SampleAttributesDS.txt
* GTEx_v7_Annotations_SubjectPhenotypesDS.txt
* GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz
* GTEx_Analysis_v7_eQTL.tar.gz

Uncompress GTEx_Analysis_v7_eQTL.tar.gz, and make sure that the contents of the resulting GTEx_Analysis_v7_eQTL directory are in the same location as the other files. In the code below, it is assumed that all data is stored in a `data` directory, at the same level as this notebook.

#### R environment
Run the following import statements to set up the environment. The `hashmap` package must be installed.

In [1]:
library(hashmap)

### Selecting samples and subsets of the data

To begin, we will explore the sample and subject annotations. Load the two tables:

In [2]:
sample.df <- read.delim("data/GTEx_v7_Annotations_SampleAttributesDS.txt", as.is=TRUE, header=TRUE, row.names=1)
subject.df <- read.delim('data/GTEx_v7_Annotations_SubjectPhenotypesDS.txt', as.is=TRUE, header=TRUE, row.names=1)

Explanations for each column header can be found in the dictionary spreadsheets:<br>
[GTEx_Analysis_v7_Annotations_SubjectPhenotypesDD.xlsx](data/GTEx_Analysis_v7_Annotations_SubjectPhenotypesDD.xlsx)<br>
[GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx](data/GTEx_Analysis_v7_Annotations_SampleAttributesDD.xlsx)

#### Subject attributes

To view the first few rows of the subject attributes, run:

In [3]:
head(subject.df)

,SEX,AGE,DTHHRDY
GTEX-1117F,2,60-69,4
GTEX-111CU,1,50-59,0
GTEX-111FC,1,60-69,1
GTEX-111VG,1,60-69,3
GTEX-111YS,1,60-69,0
GTEX-1122O,2,60-69,0


The `DTHHRDY` column encodes the death classification based on the 4-point _Hardy Scale_. The coding corresponds to the following categories:<br>
0: Ventilator Case; 1: Violent and fast death; 2: Fast death of natural causes; 3: Intermediate death; 4: Slow death

For additional details on the subject attributes, see [GTEx_Analysis_v7_Annotations_SubjectPhenotypesDD.xlsx](data/GTEx_Analysis_v7_Annotations_SubjectPhenotypesDD.xlsx).

_Note that the attribute tables from the GTEx Portal contain open-access data only; protected information (e.g., exact donor age, phenotypic information, etc.) is available from dbGaP._ Consequently, only age ranges are provided here.

#### Sample attributes

The sample attributes file contains information on all samples in the GTEx resource, including RNA-seq samples and whole genome sequence (WGS) samples.

In [4]:
head(sample.df)

,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,SMNABTCH,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
GTEX-1117F-0003-SM-58Q7G,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,BP-38516,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0003-SM-5DWSB,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,BP-38516,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0003-SM-6WBT7,NA,B1,,NA,Blood,Whole Blood,0013756,1188,NA,BP-38543,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,BP-43693,...,14579275,12025354,0.003163721,14634407,50.09436,0.003101902,0.9928265,NA,0,50.12628
GTEX-1117F-0426-SM-5EGHI,0,B1,"2 pieces, !5% fibrous connective tissue, delineated (rep)",7.1,Muscle,Muscle - Skeletal,0011907,1220,1119,BP-43495,...,13134349,11578874,0.003991152,13307871,50.32811,0.006990725,0.9942121,NA,0,49.90517
GTEX-1117F-0526-SM-5EGHJ,0,B1,"2 pieces, clean, Monckebeg medial sclerosis, rep delineated",8.0,Blood Vessel,Artery - Tibial,0007610,1221,1120,BP-43495,...,13169835,11015113,0.004285066,13160068,49.98145,0.002867028,0.9927107,NA,0,50.22785


Each GTEx release defines an analysis freeze containing samples that pass all quality controls, for each sample type. The analysis freeze column is called `SMAFRZE`. For a summary of analysis freeze samples of each data type in the V7 release, run:

In [5]:
table(sample.df['SMAFRZE'])


EXCLUDE    OMNI  RNASEQ     WES     WGS 
   2232     450   11688     593     635 

Here, we'll focus on the RNA-seq samples. To facilitate this, we define a new data frame containing the RNA-seq analysis freeze samples only:

In [6]:
rnaseq.sample.df <- sample.df[sample.df['SMAFRZE']=='RNASEQ', ]

To view the first few rows of attributes for these samples, run

In [7]:
head(rnaseq.sample.df)

,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,SMNABTCH,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
GTEX-1117F-0226-SM-5GZZ7,0,B1,"2 pieces, ~15% vessel stroma, rep delineated",6.8,Adipose Tissue,Adipose - Subcutaneous,0002190,1214,1125,BP-43693,...,14579275,12025354,0.003163721,14634407,50.09436,0.003101902,0.9928265,NA,0,50.12628
GTEX-1117F-0426-SM-5EGHI,0,B1,"2 pieces, !5% fibrous connective tissue, delineated (rep)",7.1,Muscle,Muscle - Skeletal,0011907,1220,1119,BP-43495,...,13134349,11578874,0.003991152,13307871,50.32811,0.006990725,0.9942121,NA,0,49.90517
GTEX-1117F-0526-SM-5EGHJ,0,B1,"2 pieces, clean, Monckebeg medial sclerosis, rep delineated",8.0,Blood Vessel,Artery - Tibial,0007610,1221,1120,BP-43495,...,13169835,11015113,0.004285066,13160068,49.98145,0.002867028,0.9927107,NA,0,50.22785
GTEX-1117F-0626-SM-5N9CS,1,B1,"2 pieces, up to 4mm aderent fat/nerve/vessel, delineated",6.9,Blood Vessel,Artery - Coronary,0001621,1243,1098,BP-43956,...,15148343,11624467,0.003379409,15282444,50.22033,0.005335459,0.9911747,NA,0,50.02504
GTEX-1117F-0726-SM-5GIEN,1,B1,"2 pieces, no abnormalities",6.3,Heart,Heart - Atrial Appendage,0006631,1244,1097,BP-44261,...,13583226,9262806,0.003450733,13745609,50.29709,0.030579332,0.9944781,NA,0,49.92987
GTEX-1117F-1326-SM-5EGHH,1,B1,"2 pieces, diffuse mesothelial hyperplasia; ~10% vessel/fibrous tissue (delineated)",5.9,Adipose Tissue,Adipose - Visceral (Omentum),0010414,1277,1066,BP-43495,...,13342709,9625148,0.004869216,13393827,50.09559,0.010321368,0.9891147,NA,0,50.05861


One of the key columns in the sample attribute table is `SMTSD`, which stands for 'Tissue Site Detail'. This enables the selection of samples from a specific tissue. For example, to obtain a summary of sample sizes in each tissue, run:

In [8]:
as.matrix(sort(table(rnaseq.sample.df['SMTSD']), decreasing=TRUE))

Muscle - Skeletal,564
Skin - Sun Exposed (Lower leg),473
Thyroid,446
Adipose - Subcutaneous,442
Artery - Tibial,441
Lung,427
Nerve - Tibial,414
Esophagus - Mucosa,407
Whole Blood,407
Skin - Not Sun Exposed (Suprapubic),387
Esophagus - Muscularis,370


The RNA-seq samples corresponding to a given tissue can be selected using the `SMTSD` column:

In [9]:
pancreas.sample.df = rnaseq.sample.df[rnaseq.sample.df['SMTSD']=='Pancreas', ]
head(pancreas.sample.df)

,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,SMNABTCH,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
GTEX-111CU-0526-SM-5EGHK,1,B1,"2 pieces, include 20 and 40% fat (attached and internal)",7.2,Pancreas,Pancreas,0001150,72,637,BP-43495,...,13106749,15136415,0.008058607,13257128,50.28520,0.001842467,0.9929395,NA,0,49.93632
GTEX-111YS-1226-SM-5EGGJ,1,B1,2 somewhat fragmented pieces,6.8,Pancreas,Pancreas,0001150,154,735,BP-44460,...,15480965,20021609,0.005361661,15599862,50.19127,0.003316094,0.9931882,NA,0,50.07460
GTEX-1122O-0726-SM-5GIEV,1,B1,"2 pieces, minimal attached and internal fat",6.0,Pancreas,Pancreas,0001150,55,948,BP-44261,...,21587613,25147179,0.006103872,21545407,49.95108,0.002715764,0.9938754,NA,0,50.22186
GTEX-1128S-0826-SM-5GZZI,1,B1,"2 pieces, up to 50% interstital fat, rep delineated; Islets well visualized, rep delineated. Some areas approach score 2/saponification/autolysis",6.2,Pancreas,Pancreas,0001150,842,1003,BP-43753,...,18722483,20292290,0.005499121,18640983,49.89094,0.004869585,0.9936422,NA,0,50.14596
GTEX-117YX-0226-SM-5EGH6,1,B1,"2 pieces; well dissected, no fat",7.0,Pancreas,Pancreas,0001150,80,726,BP-44460,...,15319276,18135569,0.007021304,15453346,50.21784,0.004131658,0.9927996,NA,0,49.85380
GTEX-11DXX-0926-SM-5H112,1,B1,"2 pieces, up to 20% fat, PanIN-1B, diffuse insular amyloidosis, acinar cell nodules",7.1,Pancreas,Pancreas,0001150,100,513,BP-43730,...,19865004,25588787,0.006011567,19780381,49.89327,0.002427112,0.9939556,NA,0,50.14606


All GTEx samples are indexed using the same identifier format. Importantly, the first two components of a *sample ID* correspond to the *donor ID* for that sample. For example, to obtain the donor IDs corresponding to the samples selected above:

In [10]:
rownames(pancreas.sample.df)[1:3]  # show the first 3 sample IDs

[1] "GTEX-111CU-0526-SM-5EGHK" "GTEX-111YS-1226-SM-5EGGJ"
[3] "GTEX-1122O-0726-SM-5GIEV"

run the following:

In [11]:
donor.ids <- unlist(lapply(strsplit(rownames(pancreas.sample.df), '-'), FUN=function(x){paste(x[1],x[2],sep="-")}))
donor.ids[1:3]  # show the first 3 donor IDs

[1] "GTEX-111CU" "GTEX-111YS" "GTEX-1122O"

Samples can then be split by donor attribute as follows:

In [12]:
pancreas.male.df = pancreas.sample.df[subject.df[donor.ids, 'SEX']==1, ]  # males are encoded as 1
pancreas.female.df = pancreas.sample.df[subject.df[donor.ids, 'SEX']==2, ]  # females are encoded as 2

### Loading expression data
Next, after having selected a specific set of samples, we show how to load the expression data corresponding to these samples. Gene-level expression is provided as raw read counts and in TPM (transcripts per million); here we will use the TPM data. Both tables are in GCT format, which contains two rows of header data. The first two columns contain the gene ID and gene name; since gene names are not unique, **all GTEx tables are indexed by GENCODE/ENSEMBL gene ID**.

In [13]:
tpm.df <- read.delim("data/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.Pancreas.gct.gz",
                     as.is=T, row.names=1, check.names=FALSE, skip=2)
gene.names.df <- tpm.df[, 'Description', drop=FALSE]
tpm.df <- tpm.df[, !(names(tpm.df) %in% c('Description'))]

cat(paste("Number of genes in table: ", dim(tpm.df)[1]))

Number of genes in table:  56202

The table contains data for all genes in the reference annotation:

In [14]:
head(tpm.df)

,GTEX-111CU-0526-SM-5EGHK,GTEX-111YS-1226-SM-5EGGJ,GTEX-1122O-0726-SM-5GIEV,GTEX-1128S-0826-SM-5GZZI,GTEX-117YX-0226-SM-5EGH6,GTEX-11DXX-0926-SM-5H112,GTEX-11EQ9-1026-SM-5H134,GTEX-11GSP-0426-SM-5A5KX,GTEX-11I78-0626-SM-5A5LZ,GTEX-11LCK-0226-SM-5A5M6,...,GTEX-ZT9W-0926-SM-57WFS,GTEX-ZTPG-1026-SM-5DUWP,GTEX-ZV7C-0726-SM-59HKH,GTEX-ZVP2-0726-SM-59HKY,GTEX-ZVT2-2026-SM-5NQ8Q,GTEX-ZVZP-0626-SM-59HL5,GTEX-ZYFG-0826-SM-5BC5T,GTEX-ZYW4-2126-SM-59HJ9,GTEX-ZYY3-0826-SM-5E44R,GTEX-ZZPU-0726-SM-5N9C8
ENSG00000223972.4,0.0000,0.05460,0.04141,0.06778,0.03802,0.03170,0.01529,0.06213,0.03664,0.01931,...,0.01598,0.01906,0.06286,0.10340,0.04275,0.03054,0.07476,0.10820,0.09764,0.03772
ENSG00000227232.4,3.0770,3.01700,5.28200,7.54500,3.42600,4.56800,4.84700,1.78200,3.65500,4.05800,...,3.91200,9.07500,6.90700,6.47800,4.18200,6.32800,5.20600,5.59100,6.64400,7.34400
ENSG00000243485.2,0.1564,0.12130,0.00000,0.03764,0.04222,0.03521,0.03396,0.10350,0.08139,0.00000,...,0.03549,0.12700,0.03491,0.09188,0.18990,0.06783,0.16610,0.04808,0.04338,0.00000
ENSG00000237613.2,0.0000,0.00000,0.01932,0.00000,0.02660,0.00000,0.04279,0.00000,0.12820,0.02703,...,0.00000,0.02667,0.00000,0.05789,0.00000,0.06411,0.07847,0.00000,0.00000,0.02640
ENSG00000268020.2,0.0000,0.00000,0.00000,0.06105,0.00000,0.00000,0.05507,0.00000,0.00000,0.00000,...,0.00000,0.03434,0.02831,0.03726,0.00000,0.00000,0.00000,0.00000,0.03518,0.00000
ENSG00000240361.1,0.0426,0.03303,0.02505,0.03075,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.03459,0.02852,0.00000,0.00000,0.02771,0.13570,0.03928,0.00000,0.00000


The expression of a specific gene can then be queried as

In [15]:
g <- tpm.df['ENSG00000254647.2', ]

To facilitate queries by gene name (as long as this is unique), a gene name &rarr; ID mapping can be constructed as follows. In the GCT tables, _Name_ contains the GENCODE ID, and _Description_ the gene name.

In [16]:
name.to.id <- hashmap(gene.names.df[,'Description'], rownames(gene.names.df))
id.to.name <- hashmap(rownames(gene.names.df), gene.names.df[,'Description'])
name.to.id[['INS']]

[1] "ENSG00000254647.2"

In [17]:
g <- t(tpm.df[name.to.id[['INS']],])
head(g, 10)

,ENSG00000254647.2
GTEX-111CU-0526-SM-5EGHK,1432.0
GTEX-111YS-1226-SM-5EGGJ,1317.0
GTEX-1122O-0726-SM-5GIEV,1190.0
GTEX-1128S-0826-SM-5GZZI,3515.0
GTEX-117YX-0226-SM-5EGH6,2810.0
GTEX-11DXX-0926-SM-5H112,635.0
GTEX-11EQ9-1026-SM-5H134,2744.0
GTEX-11GSP-0426-SM-5A5KX,434.5
GTEX-11I78-0626-SM-5A5LZ,9137.0
GTEX-11LCK-0226-SM-5A5M6,3038.0


Working with dataframes provides many useful functionalities. For example, the median expression across a set of samples can be simply calculated as

In [18]:
median.expr <- apply(tpm.df, 1, median)
head(median.expr)

ENSG00000223972.4 ENSG00000227232.4 ENSG00000243485.2 ENSG00000237613.2 
         0.027255          5.553000          0.034055          0.022915 
ENSG00000268020.2 ENSG00000240361.1 
         0.000000          0.000000

and the median expression for a specific gene can then be queried as

In [19]:
median.expr[name.to.id[['INS']]]

ENSG00000254647.2 
             2338

### Overview of the cis-eQTL data
In this section, we provide an overview of the cis-eQTL data. This consists of the following set of files (available from the GTEx Portal), for each tissue:
* Gene/eGene summary statistics<br>(files ending in *.egenes.txt.gz)
* The list of all significant variant-gene associations found in a tissue<br>(files ending in *.signif_variant_gene_pairs.txt.gz)
* The association results for all variant-gene pairs tested in a tissue<br>(files ending in *.allpairs.txt.gz)

Additionally, all input files used for eQTL mapping are available from the Portal, including normalized expression and covariates (both described in the tutorial).

Here, we will focus on the eGene summary and significant pair tables.

Begin by loading the gene summary table:

In [20]:
gene.df = read.delim('data/Pancreas.v7.egenes.txt.gz', as.is=TRUE, header=TRUE, row.names=1)

To view the different columns available in this table, inspect a specific row:

In [21]:
t(gene.df[name.to.id[['INS']], ])

,ENSG00000254647.2
gene_name,INS
gene_chr,11
gene_start,2181009
gene_end,2182571
strand,-
num_var,9167
beta_shape1,1.04048
beta_shape2,1457.85
true_df,163.574
pval_true_df,0.000846469


As this example shows, the table contains data for all genes, not just eGenes. To select eGenes at FDR ≤ 0.05, corresponding to the set used in GTEx Consortium analyses, run:

In [22]:
egenes.df = gene.df[gene.df['qval']<=0.05, ]
cat(paste('Number of eGenes: ', dim(egenes.df)[1]))

Number of eGenes:  7146

As a toy example, we'll next show how to extract all significant variant-gene pairs corresponding to the eGene with the strongest association in Pancreas.

In [23]:
head(egenes.df[order(egenes.df['pval_nominal']), c('variant_id', 'pval_nominal')])

,variant_id,pval_nominal
ENSG00000197728.5,12_56435929_C_G_b37,3.24344e-97
ENSG00000013573.12,12_31225344_A_G_b37,1.06518e-93
ENSG00000166435.11,11_74659302_A_T_b37,4.15005e-84
ENSG00000174652.13,19_9517537_C_G_b37,1.26526e-82
ENSG00000204792.2,2_75185856_G_T_b37,5.73833e-80
ENSG00000164308.12,5_96252589_T_C_b37,1.50095e-79


Load the association results for all significant variant-gene pairs:

In [24]:
pairs.df <- read.delim('data/Pancreas.v7.signif_variant_gene_pairs.txt.gz', as.is=TRUE, header=TRUE)

Then, select the pairs corresponding to the top gene:

In [25]:
selected.pairs.df <- pairs.df[pairs.df['gene_id']=='ENSG00000197728.5', ]

In [26]:
head(selected.pairs.df)

,variant_id,gene_id,tss_distance,ma_samples,ma_count,maf,pval_nominal,slope,slope_se,pval_nominal_threshold,min_pval_nominal,pval_beta
486741,12_56271219_T_A_b37,ENSG00000197728.5,-164418,42,42,0.0954545,2.49133e-05,-0.687914,0.1590120,4.13171e-05,3.24344e-97,9.64438e-89
486742,12_56354272_G_C_b37,ENSG00000197728.5,-81365,115,140,0.3317540,8.26861e-14,0.681209,0.0842585,4.13171e-05,3.24344e-97,9.64438e-89
486743,12_56364321_A_G_b37,ENSG00000197728.5,-71316,122,149,0.3417430,1.42473e-17,0.721492,0.0761725,4.13171e-05,3.24344e-97,9.64438e-89
486744,12_56366160_G_C_b37,ENSG00000197728.5,-69477,48,54,0.1232880,2.00943e-05,0.580996,0.1327020,4.13171e-05,3.24344e-97,9.64438e-89
486745,12_56367966_C_T_b37,ENSG00000197728.5,-67671,40,45,0.1027400,1.17099e-05,-0.577004,0.1280260,4.13171e-05,3.24344e-97,9.64438e-89
486746,12_56368078_G_A_b37,ENSG00000197728.5,-67559,130,161,0.3659090,1.05239e-27,-0.865084,0.0667007,4.13171e-05,3.24344e-97,9.64438e-89


In [27]:
cat(paste0('Significant variant-gene pairs for gene ',
          id.to.name[['ENSG00000197728.5']], ': ', dim(selected.pairs.df)[1]))

Significant variant-gene pairs for gene RPS26: 155